In [1]:
import time, sys, string
import sqlite3
import json
import datetime
import calendar
import os
import fcntl
import re
import crcmod
from binascii import unhexlify
import paho.mqtt.client as mqtt
from random import randint

def connect():
    #global client
    #client = mqtt.Client(client_id=os.environ['MQTT_CLIENT_ID'])
    #client.connect(os.environ['MQTT_SERVER'])
    try:
        global file
        global fd
        file = open('/dev/hidVoltronic', 'r+')
        fd = file.fileno()
        fl = fcntl.fcntl(fd, fcntl.F_GETFL)
        fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)
    except Exception as e:
        print('error open file descriptor: ' + str(e))
        exit()

def disconnect():
    file.close()

def serial_command(command):
    print(command)
    try:
        xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
        # Original command probably worked with earlier python versions
        #command_crc = command + unhexlify(hex(xmodem_crc_func(command)).replace('0x','',1)) + '\x0d'
        # Need to encode the commands for at least from python 3.6.
        command_a=command.encode('utf-8')
        command_b=unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1))
        command_c='\x0d'.encode('utf-8')
        
        command_crc = command_a + command_b + command_c

        os.write(fd, command_crc)

        response = b''
        timeout_counter = 0
        while response.find(b'\r') < 0:
            if timeout_counter > 500:
                raise Exception('Read operation timed out')
            timeout_counter += 1
            try:
                response += os.read(fd, 500)
            except Exception as e:
                # print("error reading response...: " + str(e))
                time.sleep(0.01)
            #if len(response) > 0 and response[0] != '(' or 'NAKss' in response:
            #    raise Exception('NAKss')

        response = response.rstrip()
        lastI = response.find(b'\r')
        response = response[1:lastI-2]
        return response
    except Exception as e:
        print('error reading inverter...: ' + str(e))
        disconnect()
        time.sleep(0.1)
        connect()
        return serial_command(command)

def get_parallel_data():
    #collect data from axpert inverter
    try:
        data = '{'
        response = serial_command('QPGS0')
        nums = response.split(' ')
        if len(nums) < 27:
            return ''

        if nums[2] == 'L':
            data += '"Gridmode":1'
        else:
            data += '"Gridmode":0'
        data += ',"SerialNumber": ' + str(int(nums[1]))
        data += ',"BatteryChargingCurrent": ' + str(int(nums[12]))
        data += ',"BatteryDischargeCurrent": ' + str(int(nums[26]))
        data += ',"TotalChargingCurrent": ' + str(int(nums[15]))
        data += ',"GridVoltage": ' + str(float(nums[4]))
        data += ',"GridFrequency": ' + str(float(nums[5]))
        data += ',"OutputVoltage": ' + str(float(nums[6]))
        data += ',"OutputFrequency": ' + str(float(nums[7]))
        data += ',"OutputAparentPower": ' + str(int(nums[8]))
        data += ',"OutputActivePower": ' + str(int(nums[9]))
        data += ',"LoadPercentage": ' + str(int(nums[10]))
        data += ',"BatteryVoltage": ' + str(float(nums[11]))
        data += ',"BatteryCapacity": ' + str(float(nums[13]))
        data += ',"PvInputVoltage": ' + str(float(nums[14]))
        data += ',"TotalAcOutputApparentPower": ' + str(int(nums[16]))
        data += ',"TotalAcOutputActivePower": ' + str(int(nums[17]))
        data += ',"TotalAcOutputPercentage": ' + str(int(nums[18]))
        # data += ',"InverterStatus": ' + nums[19]
        data += ',"OutputMode": ' + str(int(nums[20]))
        data += ',"ChargerSourcePriority": ' + str(int(nums[21]))
        data += ',"MaxChargeCurrent": ' + str(int(nums[22]))
        data += ',"MaxChargerRange": ' + str(int(nums[23]))
        data += ',"MaxAcChargerCurrent": ' + str(int(nums[24]))
        data += ',"PvInputCurrentForBattery": ' + str(int(nums[25]))
        if nums[2] == 'B':
            data += ',"Solarmode":1'
        else:
            data += ',"Solarmode":0'

        data += '}'
    except Exception as e:
        print('error parsing inverter data...: ' + str(e))
        return ''
    return data

def get_data():
    #collect data from axpert inverter
    try:
        response = serial_command('QPIGS')
        nums = response.split(' ')
        if len(nums) < 21:
            return ''

        data = '{'

        data += '"BusVoltage":' + str(float(nums[7]))
        data += ',"InverterHeatsinkTemperature":' + str(float(nums[11]))
        data += ',"BatteryVoltageFromScc":' + str(float(nums[14]))
        data += ',"PvInputCurrent":' + str(int(nums[12]))
        data += ',"PvInputVoltage":' + str(float(nums[13]))
        data += ',"PvInputPower":' + str(int(nums[19]))
        data += ',"BatteryChargingCurrent": ' + str(int(nums[9]))
        data += ',"BatteryDischargeCurrent":' + str(int(nums[15]))
        data += ',"DeviceStatus":"' + nums[16] + '"'

        data += '}'
        return data
    except Exception as e:
        print('error parsing inverter data...: ' + str(e))
        return ''

def send_data(data, topic):
    try:
        client.publish(topic, data)
    except Exception as e:
        print("error sending to emoncms...: " + str(e))
        return 0
    return 1

In [4]:
import time, sys, string
import sqlite3
import json
import datetime
import calendar
import os
import fcntl
import re
import crcmod
from binascii import unhexlify
import paho.mqtt.client as mqtt
from random import randint

def connect():
    global file
    global fd
    # Assumes that the Axpert has a udev rule defined:
    file = open('/dev/hidVoltronic', 'r+')
    fd = file.fileno()
    print('Maak Axpert oop as leer nommer')
    print(fd)

def disconnect():
    file.close()

def serial_command(command):
    print(command)
    try:
        xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
        command_a=command.encode('utf-8')
        command_b=unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1))
        command_c='\x0d'.encode('utf-8')
        
        command_crc = command_a + command_b + command_c

        os.write(fd, command_crc)

        response = b''
        timeout_counter = 0
        while response.find(b'\r') < 0:
            if timeout_counter > 500:
                raise Exception('Read operation timed out')
            timeout_counter += 1
            try:
                response += os.read(fd, 500)
            except Exception as e:
                # print("error reading response...: " + str(e))
                time.sleep(0.01)
            #if len(response) > 0 and response[0] != '(' or 'NAKss' in response:
            #    raise Exception('NAKss')

        response = response.rstrip()
        lastI = response.find(b'\r')
        response = response[1:lastI-2]
        return response
    except Exception as e:
        print('error reading inverter...: ' + str(e))
        disconnect()
        time.sleep(0.1)
        connect()
        return serial_command(command)

In [50]:
command = 'QPIGS'
xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
# Original command probably worked with earlier python versions
#command_crc = command + unhexlify(hex(xmodem_crc_func(command)).replace('0x','',1)) + '\x0d'
# Need to encode the commands for at least from python 3.6.
command_a=command.encode('utf-8')
command_b=unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1))
command_c='\x0d'.encode('utf-8')

command_crc = command_a + command_b + command_c

print(command_crc)

os.write(fd, command_crc)
response = b''

import select
r, w, e = select.select([ fd ], [], [], 0)
if fd in r:
    response += os.read(fd, 1000)

print(response)

b'QPIGS\xb7\xa9\r'
b'050 0000'


In [2]:
global file
global fd
#file = open('/dev/hidraw0', 'r+')
file = open('/dev/hidVoltronic', 'r+')
fd = file.fileno()
print(fd)
print(file)
#os.close(fd)


42
<_io.TextIOWrapper name='/dev/hidVoltronic' mode='r+' encoding='UTF-8'>


In [6]:
connect()
response = serial_command('QPIGS')
print(response.decode('utf-8'))
disconnect()


Maak Axpert oop as leer nommer
37
QPIGS
245.5 49.9 245.5 49.9 0368 0289 007 428 53.20 000 100 0050 0000 000.0 00.00 00001 01010101 00 00 00000 110


In [23]:
response = serial_command('QID')
print(response)
response = serial_command('QVFW')
print(response)
response = serial_command('QPIRI')
print(response)
response = serial_command('QFLAG')
print(response)
response = serial_command('QMOD')
print(response)
response = serial_command('QPIWS')
print(response)

QID
b'55355535553555'
QVFW
b'VERFW:00072.70'
QPIRI
b'230.0 21.7 230.0 50.0 21.7 5000 4000 48.0 46.0 46.0 53.2 53.2 2 10 040 0 0 2 9 01 0 1 48.0 0 1 000'
QFLAG
b'ExyDabjkuvz'
QMOD
b'L'
QPIWS
b'00000000000000000000000000000000'


In [24]:
# QPI wat nie reg werk nie:
response = serial_command('QPI')
print(response)

# QPI wat reg werk:
command = 'QPI'
xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')

command_a=command.encode('utf-8')
command_b=unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1))
command_c='\x0d'.encode('utf-8')

command_crc = command_a + command_b + command_c

print(command_crc)

os.write(fd, command_crc)
print(os.read(fd, 512))
#30 for Axpert KS series 

QPI
b'PI'
b'QPI\xbe\xac\r'
b'(PI30\x9a\x0b\r'


In [55]:
command = 'POP02'

xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')

command_a=command.encode('utf-8')
command_b=unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1))
command_c='\x0d'.encode('utf-8')

command_crc = command_a + command_b + command_c

print('Command a')
print(command_a)
#print(command_b)
#print(command_c)
print('Volle Command crc en newline')
print(command_crc)

print('CRC in hex')
print((hex(xmodem_crc_func(command_a))))
print('CRC in bitstring')
print(unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1)))

# POP02 seems to be an exception.  It causes the CRC to make a reserved character
# \n.  This has to be incremented.
print('Hier is die inkrementeerde string:')
print(unhexlify('e20b'))

# Vir POP00
command_crc = b'POP00\xc2H\r'
# Vir POP02
#command_crc = b'POP02\xe2\x0b\r'
os.write(fd, command_crc)

print(os.read(fd, 512))



#b'POP00\xc2H\r'
#b'POP01\xd2i\r'

Command a
b'POP02'
Volle Command crc en newline
b'POP02\xe2\n\r'
CRC in hex
0xe20a
CRC in bitstring
b'\xe2\n'
Hier is die inkrementeerde string:
b'\xe2\x0b'
b'(ACK9 \r\x00'


In [1]:
connect()

NameError: name 'connect' is not defined

In [3]:
os.close(fd)

In [17]:
# 00 for utility first, 01 for solar first, 02 for SBU priority 
xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
# Original command probably worked with earlier python versions
#command_crc = command + unhexlify(hex(xmodem_crc_func(command)).replace('0x','',1)) + '\x0d'
# Need to encode the commands for at least from python 3.6.
command_a=command.encode('utf-8')
command_b=unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1))
command_c='\x0d'.encode('utf-8')

command_crc = command_a + command_b + command_c

os.write(fd, command_crc)


response = serial_command('POP2')
print(response)

POP 02


KeyboardInterrupt: 

In [6]:
command = 'QPIGS'
command = 'QID'
command = 'QPIGS'

xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
#command_crc = command + unhexlify(hex(xmodem_crc_func(command)).replace('0x','',1)) + '\x0d'
a=command.encode('utf-8')
b=unhexlify(hex(xmodem_crc_func(a)).replace('0x','',1))
c='\x0d'.encode('utf-8')
type(a+b+c)
command_crc=a+b+c

os.write(fd, command_crc)
response = ''
response = os.read(fd, 100)

#timeout_counter = 0
#while '\r' not in response:
#    if timeout_counter > 500:
#        raise Exception('Read operation timed out')
#    timeout_counter += 1
#    try:
#        response += os.read(fd, 100)
#    except Exception as e:
#        # print("error reading response...: " + str(e))
#        time.sleep(0.01)
#    if len(response) > 0 and response[0] != '(' or 'NAKss' in response:
#        raise Exception('NAKss')

print(response)

KeyboardInterrupt: 

In [7]:
# Moet response decode gebruik vir binary string
#print(response.decode("utf-8"))
print(response[0:6].find(b'\r'))
print(response[0:6])
#disconnect()
# Dan gebruik die find funksie om die \r te kry en skryf die while lus hierbo oor.
a = b'22'
a +=response
print(a)

TypeError: must be str, not bytes

In [56]:
xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
# Original command probably worked with earlier python versions
#command_crc = command + unhexlify(hex(xmodem_crc_func(command)).replace('0x','',1)) + '\x0d'
# Need to encode the commands for at least from python 3.6.
command_a=command.encode('utf-8')
command_b=unhexlify(hex(xmodem_crc_func(command_a)).replace('0x','',1))
command_c='\x0d'.encode('utf-8')

command_crc = command_a + command_b + command_c

os.write(fd, command_crc)

response = b''
timeout_counter = 0
while response.find(b'\r') < 0:
    if timeout_counter > 500:
        raise Exception('Read operation timed out')
    timeout_counter += 1
    try:
        response += os.read(fd, 100)
    except Exception as e:
        # print("error reading response...: " + str(e))
        time.sleep(0.01)
    #if len(response) > 0 and response[0] != '(' or 'NAKss' in response:
    #    raise Exception('NAKss')

print(response)

b'(241.8 50.0 241.8 50.0 0411 0335 008 428 53.20 000 100 0048 0000 000.0 00.00 00001 00010101 00 00 00000 110\x0b\xf6\r\x00\x00'


In [8]:
#response.find(b'\r')
len(response)
print(response.decode("utf-8"))
uitsetstring = response.decode("utf-8")

nums = uitsetstring.split(' ')

data = '{'

data += '"BusVoltage":' + str(float(nums[7]))
data += ',"InverterHeatsinkTemperature":' + str(float(nums[11]))
data += ',"BatteryVoltageFromScc":' + str(float(nums[14]))
data += ',"PvInputCurrent":' + str(int(nums[12]))
data += ',"PvInputVoltage":' + str(float(nums[13]))
data += ',"PvInputPower":' + str(int(nums[19]))
data += ',"BatteryChargingCurrent": ' + str(int(nums[9]))
data += ',"BatteryDischargeCurrent":' + str(int(nums[15]))
data += ',"DeviceStatus":"' + nums[16] + '"'

data += '}'

print(data)

# Te doen:  Herskryf die bostaande module dat dit werk vir die opstelling soos dit nou 
# is met die bytestrings
# Skryf uit na 'n leer wat presies dieselfde formaat is as die ruby program.
# Maak seker dat die hidraw altyd selfde is.
# Kombineer dan met battery log program.
# Toets ook om 'n opdrag te stuur aan die inverter soos POP01

AttributeError: 'str' object has no attribute 'decode'